In [ ]:
import xlwings as xw
import sources.parts_list_0802 as pl
import pandas as pd
import os
from datetime import datetime as dt
import sources.logins as lg
import pathlib
import qrcode
from time import sleep
import xlwings as xw
from IPython.display import Image, display

In [ ]:
driver = lg.check_driver()
try:
    driver.close()
except Exception as e:
    pass

In [ ]:
d_type=''
d_type = 'all' #download for routine-> ignore #
lg.download_consolidated(d_type)

# Rename for Download All
if d_type =='all':
    folder_name = 'Downloads'
    file,ctime = lg.file_latest(folder_name)
    new_name = f'{file.split(".")[0]}_All.xls'
    fname = os.path.join(folder_name,file)
    n_name =  os.path.join(folder_name,new_name)
    try:
        os.rename(fname,n_name)
    except Exception as e:
        print (e)
lg.backups('Downloads','files','.xls')
print('Done')

In [ ]:
# initialize
a =pl.parts_list()

try:
    conn.close()
except:
    pass
intro = str(a.__doc__).split('\n')
for i in intro:
    print(i)
conn = a.question()
# pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'",conn)

In [ ]:
conn.close()

In [ ]:
name ="SPLING HOLDING BASE(1.44MM)"
q = f'''
        SELECT p.[RMA NO.],PART_NO,PART_DESCRIPTION
        
        FROM parts p
        WHERE PART_DESCRIPTION = "{name}"
    '''
pd.read_sql(q,conn)

In [ ]:
# sear customer name  and return customer code
q = '''
        SELECT DISTINCT CUSTOMER_CODE, CUSTOMER_NAME
        FROM consolidated
        ORDER BY customer_code
        
    '''

customer_list = pd.read_sql(q,conn)

cus_search = str(input('Customer name:'))
df = customer_list[customer_list['CUSTOMER_NAME'].str.contains(cus_search.title())]
df = df.reset_index()
df = df.drop(columns=['index'])
df

In [ ]:
#ff asset
ff_asset_code ="'FMSV00080','FMSV00293','FMSV'" #=> FTYO,FFAP, FFVN

In [ ]:
q = f'''
        SELECT * 
        FROM consolidated c
        
        WHERE c.customer_code in({ff_asset_code})
        ORDER BY c.[rma no.] DESC
     '''
pd.read_sql(q,conn)

In [ ]:
# add sheet to database
file = lg.file_select(folder_name='Downloads')

In [ ]:
# add intall sheet
df1 = pd.read_excel(file,sheet_name=None)
(df1['csvdata']).to_sql('install',conn,index=False,if_exists='replace')

In [ ]:
#filter by customer code
customer_code = 'FMSV00090'

q = f'''
        SELECT DISTINCT [rma no.],serial_no, model,recieve_date,[repair date], [repair size],
        [defect note],approval
        
        FROM consolidated
        WHERE customer_code='{customer_code}'
        
        ORDER BY SERIAL_NO,recieve_date
'''
raw91 = pd.read_sql(q,conn)
raw91

In [ ]:
file_name = 'Service Contract/nhietdoi_tw.xlsx'
raw91.to_excel(file_name,index = False)

In [ ]:
wb = xw.Book(file_name)
sh_new = wb.sheets.add()
sh_new.name = 'Summary'

sh =wb.sheets('Sheet1')
sm = wb.sheets('Summary')

In [ ]:
#add count by sn
lr = sh.range('A' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
lc = sh.range('A1').end('right').column
d = pl.summary_report()
rma=''
sh.cells(1,lc+1).value='Repair'
sh.cells(1,lc+2).value = 'No#'
for i in range(2,lr+1):
    rma = sh.range(f'A{i}').value
#     sh.range(f'G{i}').value = d.issues(conn,rma)
#     sh.range(f'H{i}').value = f'=COUNTIF($B$2:$B{i},B{i})'
    sh.cells(i,lc+1).value = d.issues(conn,rma)
    sh.cells(i,lc+2).value = f'=COUNTIF($B$2:$B{i},B{i})'

wb.save()
df = pd.read_excel(file_name,sheet_name='Sheet1')

In [ ]:
def part_string(rma):
    part_str =''

    q = f'''
            SELECT * from parts
            WHERE [RMA NO.]='{rma}'
        '''
    part_list = pd.read_sql(q,conn)
    for i in range(len(part_list)):
        part_str +=  part_list['PART_DESCRIPTION'][i]+ '\n'
    return part_str[:-1]
# title
sm.range('A1').value ='No.'
sm.range('B1').value ='Model'
sm.range('C1').value ='Serial'
sm.range('D1').value ='Time'
for i in range(0,max(df['No#'])):
    sm.range('E1').offset(0,i).value = i+1
# subtitle
sn_list=[]
details=['Receive','Repair','Size','Issue','RMA#','Defect','Parts','Approval']

#lay out
num = len(details)
for i in range(2,lr+1):
    model = sh.range(f'C{i}').value
    sn = sh.range(f'B{i}').value
    if sn not in sn_list:
        i_row = len(sn_list)
        sm.range('A2').offset(i_row * num).value = i_row+1
        sm.range('B2').offset(i_row * num).value = model
        sm.range('C2').offset(i_row * num).value = sn
        
        sm.range('D6').offset(i_row*5).value = 'RMA#'
        for i_detail in range(len(details)):
            sm.range('D2').offset(i_row * num + i_detail).value = details[i_detail]
        sn_list.append(sn)


In [ ]:
sm.clear_contents()

In [ ]:
sm_lrow

In [ ]:
# no need export 91
new_file_name = str(input('Save as .xlsx: '))
if not new_file_name.endswith('.xlsx'): new_file_name +='.xlsx'
new_file_name

In [ ]:
wb = xw.Book(new_file_name)
sh_new = wb.sheets.add()
sh_new.name = 'Summary'
sm = wb.sheets('Summary')

In [ ]:
# fill in data details
sm_lrow = len(sn_list) * num
for i in range(0,sm_lrow):

    sn = sm.range('C2').offset(i,0).value
    count = df[df['SERIAL_NO'] == sn]['SERIAL_NO'].count()
    if sn!=None:
#         print (i,sn,count)
#         i_row= i+2
        for j in range(0,count):
            filter_data = df.loc[(df['SERIAL_NO']==sn) & (df['No#']==j+1)]
            filter_data = filter_data.reset_index()
            sm.range('E2').offset(i,j).value = filter_data['RECIEVE_DATE'][0]
            sm.range('E3').offset(i,j).value = filter_data['Repair Date'][0]
            sm.range('E4').offset(i,j).value = filter_data['Repair Size'][0]
            sm.range('E5').offset(i,j).value = filter_data['Repair'][0]
            sm.range('E6').offset(i,j).value = filter_data['RMA No.'][0]
            sm.range('E7').offset(i,j).value = filter_data['Defect Note'][0]
            sm.range('E8').offset(i,j).value = part_string(filter_data['RMA No.'][0])
            sm.range('E9').offset(i,j).value = filter_data['Approval'][0]

In [ ]:
sm.autofit()

In [ ]:
len(details)

In [ ]:
# format
for i in range(len(sn_list)):
    rng = sm.range(sm.cells(2,1),sm.cells(1+len(details),4+max(df['No#']))).offset(i * len(details))
    #                       A2   ->       hang so details cot so max No#      
    for i in range(7,11):
        rng.api.Borders(i).LineStyle = 1
        rng.api.Borders(i).Weight = 3

# Research clearify  parts by history repair



In [ ]:
#filter by customer code
# customer_code = 'FMSV00360'
q = f'''
        SELECT DISTINCT [rma no.],serial_no, model,recieve_date,[repair date], [repair size],[defect note],repair_status
        
        FROM consolidated
        
        WHERE approval = 'Approval'
        
        and [Repair Date] > '2021-12-31'
        ORDER BY [Repair Date],SERIAL_NO,recieve_date
'''
raw91 = pd.read_sql(q,conn)
raw91

In [ ]:
# filter model eg-600wr v2
q = f'''
     SELECT distinct [RMA NO.],MODEL,serial_no
     FROM consolidated c
     WHERE c.model ='EG-600WR'
     '''
pd.read_sql(q,conn)

In [ ]:
# replace model eg-600wr v2
q = '''
        UPDATE consolidated 
        SET model = 'EG-600WR'
        WHERE model = 'EG-600WR V2';
        
    '''
conn.execute(q)

In [ ]:
#create rma list and import to 
rma_list = "'"
for rma in raw91['RMA No.']:
    rma_list = rma_list +rma+ "','"
rma_list = rma_list[:-2]


In [ ]:
# create part list column
parts_list = ('ISA','BSA','CHA','FCT','FSA','FSB','LGB','VCA')


In [ ]:

q = f'''
        SELECT DISTINCT p.[RMA No.],c.model,p.part_description,pn.short_name
        FROM (parts p
        LEFT JOIN part_name pn ON p.part_description = pn.part_description)
        LEFT JOIN consolidated c ON p.[rma no.] = c.[rma no.]
        WHERE c.[RMA NO.] IN ({rma_list})
        AND pn.short_name IN {str(parts_list)}
        
    '''
parts = pd.read_sql(q,conn)
# parts.to_excel('parts.xlsx',index = False)
parts

In [ ]:
# research parts
q = '''
        SELECT p.part_no,p.part_description,pn.short_name
        FROM parts p
        LEFT JOIN part_name pn ON p.part_description = pn.part_description
        
        WHERE pn.short_name is null

    '''
ppp = pd.read_sql(q,conn)
ppp.to_excel('part_no_short_name.xlsx',index = False)

In [ ]:
xw.Book('part_no_short_name.xlsx')

In [ ]:
q = f'''
        SELECT  *
        FROM  part_name
        WHERE short_name in {str(parts_list)}
        ORDER BY short_name
        
    '''
ppp = pd.read_sql(q,conn)
ppp

####  Write to new file

In [ ]:
wb = xw.Book()
wb.sheets.add('Summary')
sh = wb.sheets('Summary')

In [ ]:
# add title
sh.range('A1').value = 'Model'
for i in range(len(parts_list)):
    sh.range('B1').offset(0,i).value = parts_list[i]

# count model and part
i=0
for model in parts['MODEL'].unique():
    sh.range('A2').offset(i).value = model
    q =f'''
            SELECT  pn.short_name as PARTS,count(*) as COUNT
            FROM (parts p
            LEFT JOIN part_name pn ON p.part_description = pn.part_description)
            LEFT JOIN consolidated c ON p.[rma no.] = c.[rma no.]
            WHERE c.[RMA NO.] IN ({rma_list})
            AND c.model ='{model}'
            AND pn.short_name IN {str(parts_list)}
            GROUP BY pn.short_name
        '''
    aaa = pd.read_sql(q,conn)
    j = 0
    for part in parts_list:
        
        try:
            part_count = int(aaa[aaa['PARTS']==part]['COUNT'])
        except:
            part_count = 0
        
        sh.range('B2').offset(i,j).value = part_count
        j+=1
    
    i+=1

In [ ]:
sh.clear_contents()

In [ ]:
sh.autofit()

In [ ]:
sh.used_range.api.AutoFilter(Field:=1)

In [ ]:
save_name =input('Save File Name: ')

wb.save(f'{save_name}.xlsx')
wb.close()

In [ ]:
# count model and part
for model in parts['MODEL'].unique():
    
    q =f'''
            SELECT  pn.short_name as PARTS,count(*) as COUNT
            FROM (parts p
            LEFT JOIN part_name pn ON p.part_description = pn.part_description)
            LEFT JOIN consolidated c ON p.[rma no.] = c.[rma no.]
            WHERE c.[RMA NO.] IN ({rma_list})
            AND c.model ='{model}'
            AND pn.short_name IN {str(parts_list)}
            GROUP BY pn.short_name
        '''
    aaa = pd.read_sql(q,conn)
    for part in parts_list:
    
        try:
            part_count = int(aaa[aaa['PARTS']==part]['COUNT'])
        except:
            part_count = 0
        

In [ ]:
model = 'EG-530WR'

q =f'''
        SELECT  pn.short_name as PARTS,count(*) as COUNT
        FROM (parts p
        LEFT JOIN part_name pn ON p.part_description = pn.part_description)
        LEFT JOIN consolidated c ON p.[rma no.] = c.[rma no.]
        WHERE c.[RMA NO.] IN ({rma_list})
        AND c.model ='{model}'
        AND pn.short_name IN {str(parts_list)}
        GROUP BY pn.short_name
    '''
aaa = pd.read_sql(q,conn)
aaa

In [ ]:
aaa[aaa['PARTS']=='BSA']['COUNT']

In [ ]:
for model in 
for part in parts_list:
    
    try:
        part_count = int(aaa[aaa['PARTS']==part]['COUNT'])
    except:
        part_count = 0
    print(part,part_count)

#### Installation


In [ ]:
ff_asset_code = ('FMSV00080','FMSV00293','FMSV')

In [ ]:
# ALL FF ASSET
q = f'''
        SELECT DISTINCT *
        FROM install
        WHERE [customer code] IN {ff_asset_code}
    '''
ff_asset = pd.read_sql(q,conn)

# create asset items
ff_asset_items = tuple(ff_asset['Serial No.'])
# ff_asset_items

In [ ]:
# ALL FF ASSET
q = f'''
        SELECT DISTINCT *
        FROM install
        WHERE [customer code] IN {ff_asset_code}
        
    '''
ff_asset = pd.read_sql(q,conn)
ff_asset


In [ ]:
# create max rma
q = f'''
        SELECT MAX([RMA NO.]) as RMA,serial_no
        FROM consolidated c
        WHERE serial_no IN {ff_asset_items}
        GROUP BY serial_no
    '''
asset_rma = pd.read_sql(q,conn)
rma_list = tuple(asset_rma['RMA'])
rma_list

#### 

In [ ]:
file_name = os.path.join(path,'hcap2023.xlsx')
writer = pd.ExcelWriter(file_name, engine = 'xlsxwriter')

In [ ]:
path

In [ ]:
# all record for asset
q = f'''
        SELECT [rma no.],[customer_name],customer_code,serial_no, model, approval,repair_status,
        CASE
            WHEN repair_status = 'Completed' THEN
                (CASE
                    WHEN approval IN ('Approval','Inspection','NoFault') THEN 'OK'

                    ELSE 'Broken' END )
            ELSE 'Waiting for QC' END
                AS status
            
        FROM consolidated c
        WHERE [rma no.] IN {rma_list}
        ORDER BY serial_no ASC, [RMA No.] DESC
    '''
way1 = pd.read_sql(q,conn)
# way1.to_excel(writer,index = False, sheet_name='way1')
way1.to_excel('hcap2023.xlsx',index = False)

In [ ]:
install = pd.read_sql('select * from install', conn)
install.to_excel('install2023.xlsx',index = False)

### Search by Serial Number

In [ ]:
#CHECK SERIAL
sn = 'Jg361K643'
q = f'''
        SELECT [rma no.],customer_name,model,serial_no,[repair size],approval,[scope connect count],[recieve_date]
        FROM consolidated
        WHERE serial_no ='{sn.upper()}'
        
        ORDER BY [rma no.] DESC
        
    '''
pd.read_sql(q,conn)

# check History
rma ='FMSV2022050061'
pd.read_sql(f"SELECT * FROM transfers WHERE rma = '{rma}'",conn)

In [ ]:
xw.Book('fesam.xlsx')

In [ ]:
path = pathlib.Path().absolute()

In [ ]:
file_name = os.path.join(path,'fesam.xlsx')
writer = pd.ExcelWriter(file_name, engine = 'xlsxwriter')

In [ ]:
writer.close()

### FY2022


In [ ]:
q = '''
        SELECT 
    '''

In [ ]:
rmas = '''

FMSV2023030051
FMSV2023040071
FMSV2023040076
FMSV2023050003
FMSV2023050019

'''
rma_list = rmas.strip().split('\n')
# tuple(rma_list)

### TAT time


In [ ]:
from sqlite3 import connect
conn = connect('history.db')

In [ ]:
sn_list = '''
1B083K166
2G348K315
1C694K023
2G366D002
2G366D121
1S094A282
1C741K051
1C692K159
2D127K103
2G391K632
3D127K096
1C741K090
1G390K229
2G348K419
1C692K141
1C728K151
1C603K041
1G374K535
1C675K021
1C653K511
1G399K080
2G348K615
1S094A412
MC380A004
4G361K368
2C730K027
2G402K022
1S094K191
1V567K309
1V567K312
2C730K032
1G391K564
1G391K928
1G391K315
1C692K077
1C692K439
4C607A005
1G388K022
5U047K005
1C692K147
2Y196K018
2G399K002
4G361K141
2D127K107
1G399K079
8G402K627
3D127K095
1C728K438
1Y157K024
1C642K363
1C653K505
1G391K371
1C692K092
1C692K160
1G391K560




    '''

sn_list = sn_list.strip().split('\n')
len(sn_list)

In [ ]:
#check by sn and record serial
rma_record =[]
for sn in sn_list:
#     sn = 'Jg361K643'
    q = f'''
            SELECT [rma no.],customer_name,model,serial_no,[repair size],approval,[scope connect count],[recieve_date]
            FROM consolidated
            WHERE upper(serial_no) ='{sn.upper()}'

            ORDER BY [rma no.] DESC

        '''
    his_rma = pd.read_sql(q,conn)
    if len(his_rma) > 1:
        display(his_rma)

        try:
            ind = int(input('Select by index[0]: '))
            rma_record.append(his_rma['RMA No.'][0])
            rma_record.append(his_rma['RMA No.'][ind])
        except:
            ind = 0
            rma_record.append(his_rma['RMA No.'][0])
    else:
        rma_record.append(his_rma['RMA No.'][0])
    
    print(rma_record)
    

In [ ]:
#combine receive
rma_list = tuple(rma_record)
q = f'''
        SELECT  DISTINCT c.[rma no.],c.customer_name,c.model,c.serial_no,
        strftime('%Y-%m-%d',CASE 
            WHEN receive NOT NULL THEN receive
            ELSE c.recieve_date END) AS [Receive Date (IN)],
        
        CASE
            WHEN m.quotation IS NULL THEN c.in_inspect_date
            WHEN m.quotation IN ('FOC','WARRANTY') THEN c.in_inspect_date
            ELSE m.quotation  END as [Quoted Date],
        
        confirmation AS [PO rec. Date],
        m.note as [Part Order Date],
        c.[Part Select Date] AS [Part Receive Date],
        ROUND(JULIANDAY(m.note) - JULIANDAY(c.[Part Select Date])) AS [TAT Parts],
        
        strftime('%Y-%m-%d',re.[Start Time]) AS[Repair Start Date],
        ROUND(JULIANDAY(c.qc) - JULIANDAY(re.[Start Time])) AS [TAT Repair],
        strftime('%Y-%m-%d',c.qc) AS [Ship Date (OUT)],
        
        ROUND(JULIANDAY(c.qc) - JULIANDAY(receive)) AS [TAT Days Completed]
        
        FROM (consolidated c
        LEFT JOIN repair_code re ON c.[RMA NO.] = re.[RMA NO.])
        LEFT JOIN m_list m ON c.[RMA NO.] = m.rma
        WHERE c.[rma no.] IN {rma_list}
        
        ORDER BY c.[rma no.]
        
    '''

tat_repair = pd.read_sql(q,conn)
tat_repair#.to_excel('inspection\\tat_repair.xlsx',index=False)

In [ ]:
tat_repair.to_excel('inspection\\tat_repair.xlsx',index=False)

In [ ]:
xw.Book('inspection\\tat_repair.xlsx')

In [ ]:
pd.read_sql('select WORKINGDAY(strptime("2023-05-15","%Y-%m-%d"),strptime("2023-05-28","%Y-%m-%d"))',conn)

In [ ]:
q = f'''
        SELECT DISTINCT re.[rma no.],re.[Start Time],
        (re.[End Time]-re.[Start Time]) as [TAT Repair],re.[End Time]
        
        FROM repair_code re
        WHERE re.[rma no.] IN {tuple(rma_list)}
    '''
pd.read_sql(q,conn).to_excel('inspection\\tat_repair.xlsx')

### HCAP KICK-OFF


In [ ]:

q = '''SELECT * 
        
        FROM transfers
        WHERE return IS  NULL
        AND customer NOT LIKE '%FUJIFILM%'

    '''

pd.read_sql(q,conn)
# 181 items

In [ ]:
# amount of waiting confirm
q = '''SELECT COUNT(*),SUM(price) AS [Amount of Waiting confirm($)]
        FROM m_list
        WHERE status LIKE '%waiting confirm'
    '''
pd.read_sql(q,conn)

In [ ]:
# total price
q = '''
        SELECT COUNT(*),SUM(part_list_date) AS [Amount of Transfers($)]
        FROM transfers
        WHERE return is null
        AND customer NOT LIKE '%FUJIFILM%'
    '''
pd.read_sql(q,conn)

In [ ]:
# min receive
q = '''SELECT MIN(RECEIVE)
        from transfers
        WHERE return is null
        AND part_list_date not null
        '''
pd.read_sql(q,conn)

In [ ]:
# GROUP BY CUSTOMER
q = '''
        SELECT customer,round(sum(part_list_date),0) AS [Price($)]
        FROM TRANSFERS
        WHERE return is null
        AND part_list_date NOT NULL
        AND customer NOT LIKE '%FUJIFILM%'
        
        group by customer
        HAVING [price($)] >10000
        ORDER BY [price($)] DESC
    '''
pd.read_sql(q,conn)

In [ ]:
#count BY customer
q = '''SELECT customer,round(sum(part_list_date),0) AS [Price($)], count(*) AS [count]
        
        FROM transfers
        
        WHERE return is null
        AND part_list_date NOT NULL
        AND customer NOT LIKE '%FUJIFILM%'
        
        GROUP BY customer
        HAVING [count] >=3
        ORDER BY [Price($)] DESC,[count] DESC
        

'''

pd.read_sql(q,conn)

### Bach Mai


In [ ]:
customer_name = 'Bach Mai Hospital'
q = f'''SELECT model,count(*) As [Quantity], round(sum(part_list_date)) AS [Amount($)]
        FROM transfers
        
        WHERE return is null
        
        AND part_list_date NOT NULL
        AND customer = '{customer_name}'
        
        GROUP BY model
        ORDER BY [Amount($)] DESC
        
    
    '''
pd.read_sql(q,conn)

In [ ]:
model ='EG-600WR'

q = f'''SELECT rma,tf.model,sn,strftime("%d-%m-%Y",receive) as received,
                strftime("%Y",[date installed]) as[Year Installed],

                issue,round(part_list_date) AS [Dealer Price (GP 41%)],
                round(part_list_date*0.85) AS [GP 20%]
        
        FROM transfers tf
        LEFT JOIN install ins ON tf.sn = ins.[serial no.]
        
        WHERE return is null
        AND part_list_date NOT NULL
        AND customer = '{customer_name}'
        and tf.model = '{model}'
        ORDER BY [Year Installed] DESC, [Dealer Price (GP 41%)] DESC
    '''
print(f'{customer_name.upper()} - Model: {model}')
pd.read_sql(q,conn)

In [ ]:
model ='EC-600WI'

q = f'''SELECT rma,tf.model,sn,strftime("%d-%m-%Y",receive) as received,
                strftime("%Y",[date installed]) as[Year Installed],

                issue,round(part_list_date) - round(part_list_date*0.85) AS [GP 20%]
        
        FROM transfers tf
        LEFT JOIN install ins ON tf.sn = ins.[serial no.]
        
        WHERE return is null
        AND part_list_date NOT NULL
        AND customer = '{customer_name}'
        and tf.model = '{model}'
        ORDER BY [Year Installed] DESC, [GP 20%] DESC
    '''
print(f'{customer_name.upper()} - Model: {model}')
pd.read_sql(q,conn)